In [5]:
import sys

sys.path.append("..")

In [6]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from tqdm import tqdm
from datasets import load_dataset
from scipy.stats import norm, multivariate_normal
import ast

from src.vae.modeling import BetaVAE

seed = 42
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Define Tag Categories

Define all possible tags for each category based on the dataset.

In [7]:
TAXONOMY = json.load(open("../data/concepts_to_tags.json", "r"))

CATEGORIES = list(TAXONOMY.keys())

# Reverse map for easy lookup (tag -> category)
TAG_TO_CATEGORY = {}
for cat, tags in TAXONOMY.items():
    for tag in tags:
        TAG_TO_CATEGORY[tag] = cat


In [8]:
tag_to_idx = {}
idx_to_tag = {}
cat_ranges = {} # Stores start/end index for each category

current_idx = 0
for cat in CATEGORIES:
    start = current_idx
    for tag in TAXONOMY[cat]:
        tag_to_idx[tag] = current_idx
        idx_to_tag[current_idx] = (cat, tag)
        current_idx += 1
    cat_ranges[cat] = (start, current_idx)

TOTAL_INPUT_DIM = current_idx
print(f"Total Input Dimension: {TOTAL_INPUT_DIM}")

Total Input Dimension: 200


## Generate tags

In [9]:
df = pd.read_csv("../data/mtg_jamendo/autotagging_top50tags_processed_cleaned.csv")
df['aspect_list'] = df['aspect_list'].apply(ast.literal_eval)
df['instrument_tags'] = df['instrument_tags'].apply(ast.literal_eval)
df['genre_tags'] = df['genre_tags'].apply(ast.literal_eval)
df['mood_tags'] = df['mood_tags'].apply(ast.literal_eval)
df

,id,tags,genre_tags,mood_tags,instrument_tags,aspect_list
0,track_0007391,"['genre---electronic', 'genre---pop', 'instrum...","[electronic, pop]",[emotional],"[bass, drums, guitar, keyboard]","[drums, bass, guitar, electronic, emotional, p..."
1,track_0015161,"['genre---instrumentalpop', 'genre---pop', 'ge...","[pop, rock]",[emotional],"[bass, drums]","[drums, bass, rock, emotional, pop]"
2,track_0015166,"['genre---dance', 'genre---electronic', 'genre...","[dance, electronic, pop, techno]",[emotional],[bass],"[bass, electronic, dance, techno, emotional, pop]"
3,track_0015167,"['genre---chillout', 'genre---easylistening', ...","[electronic, pop]",[emotional],"[bass, violin]","[bass, electronic, emotional, pop, violin]"
4,track_0015169,"['genre---electronic', 'genre---instrumentalpo...","[electronic, pop]",[emotional],"[bass, drums]","[drums, bass, electronic, emotional, pop]"
...,...,...,...,...,...,...
2036,track_1420702,"['genre---dance', 'genre---easylistening', 'ge...",[dance],"[funk, happy]","[bass, drums, keyboard]","[drums, bass, dance, funk, keyboard, happy]"
2037,track_1420704,"['genre---dance', 'genre---easylistening', 'in...",[dance],[happy],"[bass, drums, keyboard]","[drums, bass, dance, keyboard, happy]"
2038,track_1420705,"['genre---dance', 'genre---easylistening', 'in...",[dance],[happy],"[bass, drums, keyboard]","[drums, bass, dance, keyboard, happy]"
2039,track_1420706,"['genre---dance', 'genre---easylistening', 'in...",[dance],[happy],"[bass, drums, keyboard]","[drums, bass, dance, keyboard, happy]"


In [12]:
input_dim = TOTAL_INPUT_DIM
latent_dim = 128
hidden_dim = 512
dropout_p = 0.25
use_batch_norm = False
beta = 0.25

model = BetaVAE(input_dim, latent_dim, hidden_dim, dropout_p, use_batch_norm, beta).to(device)
model.load_state_dict(torch.load("../models/vae_final.pth", map_location=device))
model.eval()

print("✓ Best model loaded successfully")

✓ Best model loaded successfully


In [13]:
def generate_tags(model, seed_tags, requests, temperature=1.0):
    """
    seeds: List of tags we ALREADY have (e.g. ['rock', 'guitar'])
    requests: Dict of how many tags we want per category (e.g. {'instrument': 2, 'mood': 1})
    """
    model.eval()
    device = next(model.parameters()).device
    
    # 1. Build the Input Vector from Seeds
    input_vec = torch.zeros(1, TOTAL_INPUT_DIM).to(device)
    
    
    # Fill in the knowns
    for tag in seed_tags:
        if tag in tag_to_idx:
            input_vec[0, tag_to_idx[tag]] = 1.0
        else:
            print(f"Warning: Seed tag '{tag}' not in taxonomy.")

    with torch.no_grad():
        # 2. Encode to get the Latent Vibe (z)
        # Note: We don't use dropout here; we want the model to use all clues we gave it.
        mu, logvar = model.encode(input_vec)
        z = model.reparameterize(mu, logvar)
        
        # 3. Decode to get probabilities for EVERYTHING
        # Output shape: [1, Total_Dim] (Values 0.0 to 1.0)
        probs = model.decode(z, temperature=temperature)[0] 
        
        # 4. Extract Top-K for requested categories
        results = {}
        
        for category, count in requests.items():
            if count <= 0:
                results[f"generated_{category}_tags"] = []
                continue

            start, end = cat_ranges[category]
            
            # Slice the probabilities relevant to this category
            cat_probs = probs[start:end]
            
            # Get Top K indices for this slice
            # We ask for count + len(seeds) just in case the model predicts the seed tag again
            top_k_vals, top_k_indices = torch.topk(cat_probs, k=count + 5)
            
            # Convert slice-indices back to global-indices, then to strings
            found_tags = []
            for i in range(len(top_k_indices)):
                local_idx = top_k_indices[i].item()
                global_idx = start + local_idx
                tag_name = idx_to_tag[global_idx][1]
                
                # Don't return tags we already provided as seeds
                if tag_name not in seed_tags:
                    found_tags.append(tag_name)
                
                if len(found_tags) == count:
                    break
            
            results[f"generated_{category}_tags"] = found_tags
            
    return results

In [14]:
def generate_tags_from_latent(model, latent_vector, requests, temperature=1.0):
    """
    Generate tags directly from a latent vector.
    
    Args:
        model: Trained VAE model
        latent_vector: torch tensor of shape (1, latent_dim) or (latent_dim,)
        requests: Dict of how many tags we want per category (e.g. {'instrument': 2, 'mood': 1})
        temperature: Temperature for sampling (higher = more diverse)
        
    Returns:
        Dict with generated tags per category
    """
    model.eval()
    device = next(model.parameters()).device
    
    # Ensure latent vector has correct shape
    if latent_vector.dim() == 1:
        latent_vector = latent_vector.unsqueeze(0)
    
    latent_vector = latent_vector.to(device)
    
    with torch.no_grad():
        # Decode latent vector to get probabilities for all tags
        probs = model.decode(latent_vector, temperature=temperature)[0]
        
        # Extract Top-K for requested categories
        results = {}
        
        for category, count in requests.items():
            if count <= 0:
                results[f"generated_{category}_tags"] = []
                continue
            
            start, end = cat_ranges[category]
            
            # Slice the probabilities relevant to this category
            cat_probs = probs[start:end]
            
            # Get Top K indices for this slice
            top_k_vals, top_k_indices = torch.topk(cat_probs, k=min(count + 5, end - start))
            
            # Convert slice-indices back to global-indices, then to strings
            found_tags = []
            for i in range(len(top_k_indices)):
                local_idx = top_k_indices[i].item()
                global_idx = start + local_idx
                tag_name = idx_to_tag[global_idx][1]
                found_tags.append(tag_name)
                
                if len(found_tags) == count:
                    break
            
            results[f"generated_{category}_tags"] = found_tags
    
    return results


In [15]:
CATEGORIES = [
    "tempo",
    "genre",
    "mood",
    "instrument"
]
N_CATEGORIES = len(CATEGORIES)
N_SAMPLES_TO_GENERATE = len(df)

# --- 1. SYNTHETIC DATA GENERATION (Replace with your actual data) ---
# We simulate a dataset where tag counts are discrete and correlated.
# Max counts are defined for simulation purposes.
MAX_COUNTS = {
    "tempo": 6,
    "genre": 6, 
    "mood": 8, 
    "instrument": 6, 
}
MEANS = {
    "tempo": 1.24,
    "genre": 1.25, 
    "mood": 1.75, 
    "instrument": 1.96,
}
VARIANCES = {
    "tempo": 0.58,
    "genre": 0.65,
    "mood": 1.07,
    "instrument": 1.12,
}

In [16]:
def generate_synthetic_correlated_data(n_records):
    """
    Creates synthetic discrete count data that serves as the 'real' dataset.
    This step is highly important: it determines the statistics (R and ECDFs)
    that the Copula will try to match.
    """
    print("--- 1. Generating Synthetic Data ---")

    # Define the desired correlation matrix (e.g., high correlation between Genre and Instrument)
    # This represents your calculated correlation matrix R.
    correlation_matrix = np.array([
        [1.0, 0.15, 0.046, -0.056],  # Tempo
        [0.15, 1.0, 0.021, -0.087],  # Genre
        [0.046, 0.021, 1.0, -0.099],  # Mood
        [-0.056, -0.087, -0.099, 1.0]   # Instrument
    ])

    # Generate correlated continuous data (Multivariate Normal)
    mean = np.zeros(N_CATEGORIES)
    z_continuous = multivariate_normal.rvs(mean=mean, cov=correlation_matrix, size=n_records)

    data = np.zeros((n_records, N_CATEGORIES), dtype=int)
    
    # Transform continuous data into discrete counts based on desired marginals
    # (using inverse CDF of an arbitrary discrete distribution for simulation)
    # This simulates your real-world data having specific tag count distributions
    for i, cat in enumerate(CATEGORIES):
        max_c = MAX_COUNTS[cat]
        # Simulate log normal-like distribution for counts
        mu = np.log(MEANS[cat]**2 / np.sqrt(MEANS[cat]**2 + VARIANCES[cat]))
        sigma = np.sqrt(np.log(1 + VARIANCES[cat] / MEANS[cat]**2))
        # Create discrete probability distribution
        x = np.arange(1, max_c + 1)
        p = (1 / (x * sigma * np.sqrt(2 * np.pi)))
        p *= np.exp(- (np.log(x) - mu)**2 / (2 * sigma**2))
        p /= p.sum()  # Normalize to sum to 1
        
        # Convert continuous z (uniform quantile) to discrete count (inverse CDF)
        uniform_quantiles = norm.cdf(z_continuous[:, i])
        
        # Quantile mapping for a simple discrete distribution
        counts = np.digitize(uniform_quantiles, np.cumsum(p[:-1])) + 1
        data[:, i] = np.clip(counts, 1, max_c)

    print(f"Synthetic Data Shape: {data.shape}")
    print(f"Calculated Correlation of Synthetic Data:\n{np.corrcoef(data.T).round(2)}")
    return data, correlation_matrix

In [17]:
data, _ = generate_synthetic_correlated_data(N_SAMPLES_TO_GENERATE)
print(data)

--- 1. Generating Synthetic Data ---
Synthetic Data Shape: (2041, 4)
Calculated Correlation of Synthetic Data:
[[ 1.    0.05  0.03 -0.03]
 [ 0.05  1.    0.04 -0.05]
 [ 0.03  0.04  1.   -0.08]
 [-0.03 -0.05 -0.08  1.  ]]
[[1 1 1 1]
 [1 1 1 1]
 [1 1 2 2]
 ...
 [1 2 1 2]
 [1 2 1 2]
 [1 1 2 1]]


In [18]:
def sample_random_latent_vectors(model, n_samples, seed=None):
    """
    Sample random latent vectors from standard normal distribution.
    
    Args:
        model: Trained VAE model
        n_samples: Number of latent vectors to sample
        seed: Random seed for reproducibility
        
    Returns:
        torch tensor of shape (n_samples, latent_dim)
    """
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    # Get latent dimension from model
    latent_dim = model.fc2_mu.out_features
    
    # Sample from standard normal distribution
    z = torch.randn(n_samples, latent_dim)
    
    return z

In [19]:
temperatures = [0.5, 0.75, 1.0, 1.0, 1.25, 1.5, 2.0]


results = []
for temp in tqdm(temperatures, desc="Temperatures"):
    for idx in tqdm(range(N_SAMPLES_TO_GENERATE), desc="Samples", leave=False):
        tags_per_category = {
            "tempo": data[idx, 0],
            "genre": data[idx, 1],
            "mood": data[idx, 2],
            "instrument": data[idx, 3],
        }

        row = df.iloc[idx]
        seed_tags = []

        for category in ['genre', 'instrument', 'mood']:
            if len(row[f"{category}_tags"]) > 1:
                seed_tags.append(np.random.choice(row[f"{category}_tags"]))
                tags_per_category[category] = tags_per_category.get(category, 1) - 1
            
        generated_tags = generate_tags(model, seed_tags, tags_per_category, temperature=temp)
        _generated_tags = []
        for gtags in generated_tags.values():
            _generated_tags.extend(gtags)

        df_entry = {
            'id': row['id'],
            'aspect_list': seed_tags + _generated_tags,
            'original_aspect_list': row['aspect_list'],
            'temperature': temp,
            **generated_tags
        }
        results.append(df_entry)

z_samples = sample_random_latent_vectors(model, N_SAMPLES_TO_GENERATE * len(temperatures), seed=seed)
# Add random latent vector generation for variety
for idx in tqdm(range(N_SAMPLES_TO_GENERATE * len(temperatures)), desc="Samples", leave=False):
    idx %= N_SAMPLES_TO_GENERATE  # Wrap around to existing data indices
    z = z_samples[idx:idx+1]  # Keep batch dimension

    num_tags_for_category = {
        "tempo": data[idx, 0],
        "genre": data[idx, 1],
        "mood": data[idx, 2],
        "instrument": data[idx, 3],
    }
    
    # Generate tags from this latent vector
    generated_tags_dict = generate_tags_from_latent(
        model, 
        z, 
        num_tags_for_category, 
        temperature=temp
    )
    
    # Flatten all generated tags
    all_tags = []
    for tag_list in generated_tags_dict.values():
        all_tags.extend(tag_list)
    
    df_entry = {
        'id': df.iloc[idx]['id'],
        'aspect_list': all_tags,
        'original_aspect_list': [],
        'temperature': temp,
        **generated_tags_dict
    }
    
    results.append(df_entry)

Temperatures:   0%|          | 0/7 [00:00<?, ?it/s]

Temperatures: 100%|██████████| 7/7 [00:10<00:00,  1.51s/it]


In [20]:
res_df = pd.DataFrame(results)
res_df

,id,aspect_list,original_aspect_list,temperature,generated_tempo_tags,generated_genre_tags,generated_mood_tags,generated_instrument_tags
0,track_0007391,"[pop, guitar, mid tempo, fun]","[drums, bass, guitar, electronic, emotional, p...",0.5,[mid tempo],[],[fun],[]
1,track_0015161,"[rock, drums, medium to uptempo, hard rock]","[drums, bass, rock, emotional, pop]",0.5,[medium to uptempo],[],[hard rock],[]
2,track_0015166,"[techno, uptempo, playful, weird, acoustic gui...","[bass, electronic, dance, techno, emotional, pop]",0.5,[uptempo],[],"[playful, weird]","[acoustic guitar, no voices]"
3,track_0015167,"[electronic, bass, slow tempo, moderate tempo,...","[bass, electronic, emotional, pop, violin]",0.5,"[slow tempo, moderate tempo]",[alternative rock],[eerie],"[male voice, electronic drums, percussion, str..."
4,track_0015169,"[pop, bass, slow tempo, cinematic, emotional, ...","[drums, bass, electronic, emotional, pop]",0.5,[slow tempo],[cinematic],"[emotional, dramatic, passionate]",[piano]
...,...,...,...,...,...,...,...,...
28569,track_1420702,"[medium tempo, soul, epic, e-bass]",[],2.0,[medium tempo],[soul],[epic],[e-bass]
28570,track_1420704,"[medium tempo, dance, energetic, acoustic guit...",[],2.0,[medium tempo],[dance],[energetic],"[acoustic guitar, percussion]"
28571,track_1420705,"[moderate tempo, alternative rock, ballad, emo...",[],2.0,[moderate tempo],"[alternative rock, ballad]",[emotional],"[tambourine, acoustic guitar]"
28572,track_1420706,"[syncopated snare, rock, reggae, happy, shimme...",[],2.0,[syncopated snare],"[rock, reggae]",[happy],"[shimmering shakers, flat male vocal]"


In [21]:
# Sort aspect list column and deduplicate tag combinations
res_df['aspect_list'] = res_df['aspect_list'].apply(lambda x: sorted(list(set(x))))
res_df = res_df.drop_duplicates(subset=['aspect_list']).reset_index(drop=True)
res_df

,id,aspect_list,original_aspect_list,temperature,generated_tempo_tags,generated_genre_tags,generated_mood_tags,generated_instrument_tags
0,track_0007391,"[fun, guitar, mid tempo, pop]","[drums, bass, guitar, electronic, emotional, p...",0.5,[mid tempo],[],[fun],[]
1,track_0015161,"[drums, hard rock, medium to uptempo, rock]","[drums, bass, rock, emotional, pop]",0.5,[medium to uptempo],[],[hard rock],[]
2,track_0015166,"[acoustic guitar, no voices, playful, techno, ...","[bass, electronic, dance, techno, emotional, pop]",0.5,[uptempo],[],"[playful, weird]","[acoustic guitar, no voices]"
3,track_0015167,"[alternative rock, bass, eerie, electronic, el...","[bass, electronic, emotional, pop, violin]",0.5,"[slow tempo, moderate tempo]",[alternative rock],[eerie],"[male voice, electronic drums, percussion, str..."
4,track_0015169,"[bass, cinematic, dramatic, emotional, passion...","[drums, bass, electronic, emotional, pop]",0.5,[slow tempo],[cinematic],"[emotional, dramatic, passionate]",[piano]
...,...,...,...,...,...,...,...,...
16039,track_1420702,"[e-bass, epic, medium tempo, soul]",[],2.0,[medium tempo],[soul],[epic],[e-bass]
16040,track_1420704,"[acoustic guitar, dance, energetic, medium tem...",[],2.0,[medium tempo],[dance],[energetic],"[acoustic guitar, percussion]"
16041,track_1420705,"[acoustic guitar, alternative rock, ballad, em...",[],2.0,[moderate tempo],"[alternative rock, ballad]",[emotional],"[tambourine, acoustic guitar]"
16042,track_1420706,"[flat male vocal, happy, reggae, rock, shimmer...",[],2.0,[syncopated snare],"[rock, reggae]",[happy],"[shimmering shakers, flat male vocal]"


In [22]:
# Add surrogate key based on track_id, original_tags and temperature
import hashlib
def generate_surrogate_key(track_id: str, original_tags: str, temperature: float) -> str:
    key_str = f"{track_id}_{original_tags}_{temperature}"
    return hashlib.md5(key_str.encode()).hexdigest()

res_df['surrogate_key'] = res_df.apply(lambda row: generate_surrogate_key(row['id'], row['original_aspect_list'], row['temperature']), axis=1)
res_df.drop(columns=['id'], inplace=True)
res_df.rename(columns={'surrogate_key': 'id'}, inplace=True)
res_df

,aspect_list,original_aspect_list,temperature,generated_tempo_tags,generated_genre_tags,generated_mood_tags,generated_instrument_tags,id
0,"[fun, guitar, mid tempo, pop]","[drums, bass, guitar, electronic, emotional, p...",0.5,[mid tempo],[],[fun],[],0ef76672c66c235284afd7850dcf4376
1,"[drums, hard rock, medium to uptempo, rock]","[drums, bass, rock, emotional, pop]",0.5,[medium to uptempo],[],[hard rock],[],2f7e73a70eb327c17c58e076885c5fc2
2,"[acoustic guitar, no voices, playful, techno, ...","[bass, electronic, dance, techno, emotional, pop]",0.5,[uptempo],[],"[playful, weird]","[acoustic guitar, no voices]",e7b23d6a80cb7de88b177814872101e2
3,"[alternative rock, bass, eerie, electronic, el...","[bass, electronic, emotional, pop, violin]",0.5,"[slow tempo, moderate tempo]",[alternative rock],[eerie],"[male voice, electronic drums, percussion, str...",843d4cb4da97694648357b6b3e82e1b8
4,"[bass, cinematic, dramatic, emotional, passion...","[drums, bass, electronic, emotional, pop]",0.5,[slow tempo],[cinematic],"[emotional, dramatic, passionate]",[piano],ad001e2982f850b70a620da461ded2af
...,...,...,...,...,...,...,...,...
16039,"[e-bass, epic, medium tempo, soul]",[],2.0,[medium tempo],[soul],[epic],[e-bass],46d273d61c3d1de4e1d7ebd66a1d7e6e
16040,"[acoustic guitar, dance, energetic, medium tem...",[],2.0,[medium tempo],[dance],[energetic],"[acoustic guitar, percussion]",0a8dee65f501cbc82485539a72f46c35
16041,"[acoustic guitar, alternative rock, ballad, em...",[],2.0,[moderate tempo],"[alternative rock, ballad]",[emotional],"[tambourine, acoustic guitar]",a02f07bf39a5cc9a3a79d29a47c374d0
16042,"[flat male vocal, happy, reggae, rock, shimmer...",[],2.0,[syncopated snare],"[rock, reggae]",[happy],"[shimmering shakers, flat male vocal]",bebe30b801c9c1723d19ae20838bfd1e


## Push to Hugginface Hub

In [23]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(res_df, test_size=0.1, random_state=42)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=42)

In [24]:
from pathlib import Path

# Create output directory
output_dir = Path("../data/vae-tags-dataset")
output_dir.mkdir(parents=True, exist_ok=True)

df_train.to_csv(output_dir / "train.csv", index=False)
df_valid.to_csv(output_dir / "validation.csv", index=False)
df_test.to_csv(output_dir / "test.csv", index=False)
all_df = pd.concat([df_train, df_valid, df_test])
all_df.to_csv(output_dir / "all.csv", index=False)

In [25]:
data_files = {
    "train": str(output_dir / "train.csv"),
    "validation": str(output_dir / "validation.csv"),
    "test": str(output_dir / "test.csv")
}
dataset = load_dataset("csv", data_files=data_files)
dataset.push_to_hub("bsienkiewicz/vae-tags-dataset", private=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/814 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/bsienkiewicz/vae-tags-dataset/commit/ad1735079989280b41138a2da0d56d110578db92', commit_message='Upload dataset', commit_description='', oid='ad1735079989280b41138a2da0d56d110578db92', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/bsienkiewicz/vae-tags-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='bsienkiewicz/vae-tags-dataset'), pr_revision=None, pr_num=None)